In [1]:
from metafid.deriv import Pricing, OptionStrategy
from metafid.data import TSETMC
import pandas as pd
import numpy as np
import jdatetime as jdt
import requests
from collections import namedtuple
from itertools import combinations
from operator import add
import re 
import finpy_tse as fpy
import psycopg
import jdatetime as jdt
import numpy as np
import re
from CONFIG import DB_PASS as PASS

In [2]:
drop_cols = ["isin","time","open","close","no","volume","low","high","y_final","eps","base_vol","unknown1","unknown2","sector","day_ul","day_ll", "share_no", "mkt_id", "sell_no", "buy_no"]
ua_list = ["شپنا","خودرو","فملي","شستا","وبصادر","فولاد","خگستر","سلام","دي","سرو","خاور","وبملت","خساپا","فخوز","فرابورس","اهرم","حسير","خپارس","هاي وب","چكاپا","توسن","وتجارت","ذوب","بساما","شبندر","سمگا","وخاور","شتران","كاريس","ت ثاميد","پالايش","لبخند","تپسي","كارا"]

In [3]:
bs = Pricing()
tsetmc = TSETMC(ua=ua_list, drop_cols=drop_cols)
now = jdt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
now

'1402-01-03 02:24:08'

In [88]:
"https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/"

'https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/'

In [6]:
def query_df(cols, table):
    with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
        with conn.cursor() as cur:
            cur.execute(f"SELECT {cols} FROM {table}")
            cur.fetchone()
            conn.commit()
            return pd.DataFrame([i[0] for i in cur], columns=re.findall("[a-z A-Z]+", cols))
sigma_df = query_df("(ua,sigma)", "sigma")

In [11]:
df = tsetmc.option_mv()
def option_type(x):
    return "call" if x.startswith("ض") else "put"
df = df.assign(type=df.symbol.map(option_type))
df = df.merge(sigma_df, on="ua", how="inner")
df["bs"] = df.apply(lambda x: round(bs.black_scholes(s_0=x["ua_final"], k=x["strike_price"], t=x["t"],sigma=float(x["sigma"]),type_=x["type"]),0),axis=1)
call = df[df.type =="call"]
put = df[df.type =="put"]


In [131]:
def same_strike_and_ex_date_on_call_put(call, put):
    cols = [i for i in call.columns if not i.startswith("ua_") and i not in ["t", "sigma", "dt", "type"]]
    def cols_(x):
        if (x.endswith("_x")) and (x.startswith("ua_")):
            return x.replace("_x","")
        elif (x.endswith("_x")):
            return "call_" + x.replace("_x","")
        elif (x.endswith("_y")):
            return "put_" + x.replace("_y","")
        else:
            return x
    df = call.merge(put[cols], on=["ua", "strike_price", "ex_date"], how="inner")
    df.columns = list(map(cols_, df.columns))
    return df
